In [5]:
import pandas as pd 
import matplotlib.pyplot as plt
import json

In [2]:
def plot_line_and_polygon(shape: pd.DataFrame, points: pd.DataFrame = None) -> None:
    fig, ax = plt.subplots(figsize=(9,13))

    if points is not None:
        ax.plot(points.Easting,points.Northing,color='red',markersize=3,label='Observed Data')
        
    ax.plot(shape.Easting,shape.Northing,linestyle='dashed',color='black',label='Magnetic Body')
    ax.scatter(shape.Easting,shape.Northing,color='blue',label='Verticies')

    ax.legend(loc='upper right')
    plt.savefig('./shape.png')

In [3]:
def save_json(all_shapes, filename="./data/shape_data.json"):
    """Save the coordinates of multiple shapes to a JSON file."""
    json_data = {"shapes": all_shapes}
    
    # Serializing json
    json_object = json.dumps(json_data, indent=4)
    
    # Writing to a JSON file
    with open(filename, "w") as outfile:
        outfile.write(json_object)

    print('json saved to: {}\n'.format(outfile.name))

In [ ]:
pts = pd.read_csv("/home/jovyan/Cproject/data/observed_data/points_anom_a_cone_1.csv",usecols=[0,1,2],header=None,sep='[\\s,]',names=['Easting','Northing','Mag'],engine='python')
print('\nFile head: \n',pts.head())

In [ ]:
shape = pd.read_csv('/home/jovyan/Cproject/data/shape_data/qgis_cone1_shape.csv',skiprows=1,usecols=[0,1],sep='[\\s,]',names=['Easting','Northing'],engine='python')
print('\nFile head: \n',shape.head())

In [ ]:
plot_line_and_polygon(shape=shape,points=pts)

In [ ]:
all_shapes = []  # List to hold all shape data

#assuming single shape
shape_data = {
    'name': f"shape 1",
    'x': shape.Easting.to_list(),
    'y': shape.Northing.to_list(),
    'magnetic_inclination': -67,
    'magnetic_declination': 177,
    'magnetic_intensity': 1,
    'top': 10,
    'bottom': 50
}

all_shapes.append(shape_data)

# Save all shapes data to JSON
save_json(all_shapes,filename='/home/jovyan/Cproject/data/shape_data/cone1_shape.json')

In [49]:
import numpy as np
from netCDF4 import Dataset
import os
import csv

In [37]:
# Open the NetCDF file
nc_file = '/home/jovyan/Cproject/data/observed_data/surface_anom_a_cone_1.nc'
ds = Dataset(nc_file)

# Extract latitude (y) and longitude (x) values
y = ds.variables['y'][:]  # Adjust to the correct variable name for latitudes
x = ds.variables['x'][:]  # Adjust to the correct variable name for longitudes

# Close the dataset to free resources
ds.close()

In [44]:
# Prepare a list to store the data grouped by center
grouped_data = []

# Iterate over each grid cell (without using meshgrid)
for i in range(len(y)-1):
    for j in range(len(x)-1):
        # The four corners are based on the adjacent grid points
        bottom_left_y = y[i]
        bottom_left_x = x[j]
        
        bottom_right_y = y[i]
        bottom_right_x = x[j+1]
        
        top_left_y = y[i+1]
        top_left_x = x[j]
        
        top_right_y = y[i+1]
        top_right_x = x[j+1]
        
        # Calculate the center of the grid cell
        center_y = (y[i] + y[i+1]) / 2
        center_x = (x[j] + x[j+1]) / 2
        
        # Create a dictionary to store the center and its corners
        cell_data = {
            'center': {'y': center_y, 'x': center_x},
            'corners': [
                {'y': bottom_left_y, 'x': bottom_left_x},
                {'y': bottom_right_y, 'x': bottom_right_x},
                {'y': top_left_y, 'x': top_left_x},
                {'y': top_right_y, 'x': top_right_x}
            ]
        }
        
        # Append the cell data to the grouped_data list
        grouped_data.append(cell_data)

In [45]:
# Display the grouped data
for idx, cell in enumerate(grouped_data):
    print(f"Cell {idx + 1}:")
    print(f"  Center: Y: {cell['center']['y']}, X: {cell['center']['x']}")
    for i, corner in enumerate(cell['corners']):
        print(f"  Corner {i + 1}: Y: {corner['y']}, X: {corner['x']}")
    print("-" * 40)

Cell 1:
  Center: Y: 4053860.5984848486, X: 545114.4927536232
  Corner 1: Y: 4053848.0, X: 545102.0
  Corner 2: Y: 4053848.0, X: 545126.9855072463
  Corner 3: Y: 4053873.196969697, X: 545102.0
  Corner 4: Y: 4053873.196969697, X: 545126.9855072463
----------------------------------------
Cell 2:
  Center: Y: 4053860.5984848486, X: 545139.4782608696
  Corner 1: Y: 4053848.0, X: 545126.9855072463
  Corner 2: Y: 4053848.0, X: 545151.9710144928
  Corner 3: Y: 4053873.196969697, X: 545126.9855072463
  Corner 4: Y: 4053873.196969697, X: 545151.9710144928
----------------------------------------
Cell 3:
  Center: Y: 4053860.5984848486, X: 545164.463768116
  Corner 1: Y: 4053848.0, X: 545151.9710144928
  Corner 2: Y: 4053848.0, X: 545176.9565217391
  Corner 3: Y: 4053873.196969697, X: 545151.9710144928
  Corner 4: Y: 4053873.196969697, X: 545176.9565217391
----------------------------------------
Cell 4:
  Center: Y: 4053860.5984848486, X: 545189.4492753623
  Corner 1: Y: 4053848.0, X: 545176.

In [50]:
# Create the directory to store the CSV files if it doesn't exist
output_dir = '/home/jovyan/Cproject/grid_shapes/'
os.makedirs(output_dir, exist_ok=True)

# Iterate over each grid cell (without using meshgrid)
for i in range(len(y)-1):
    for j in range(len(x)-1):
        # The four corners are based on the adjacent grid points
        bottom_left_y = y[i]
        bottom_left_x = x[j]
        
        bottom_right_y = y[i]
        bottom_right_x = x[j+1]
        
        top_left_y = y[i+1]
        top_left_x = x[j]
        
        top_right_y = y[i+1]
        top_right_x = x[j+1]
        
        # Prepare the corner data for this grid cell
        corner_data = [
            [bottom_left_x, bottom_left_y],  # Bottom-left corner
            [bottom_right_x, bottom_right_y],  # Bottom-right corner
            [top_left_x, top_left_y],  # Top-left corner
            [top_right_x, top_right_y]  # Top-right corner
        ]
        
        # Define a unique filename for each set of corners based on the grid position
        output_file = os.path.join(output_dir, f'corner_cell_{i}_{j}.csv')
        
        # Write the corner data to the CSV file (without header)
        with open(output_file, 'w', newline='') as corner_file:
            writer = csv.writer(corner_file)
            # Directly write the corner data without headers
            writer.writerows(corner_data)
        
        print(f"Corners for cell ({i}, {j}) have been successfully exported to {output_file}")


Corners for cell (0, 0) have been successfully exported to /home/jovyan/Cproject/grid_shapes/corner_cell_0_0.csv
Corners for cell (0, 1) have been successfully exported to /home/jovyan/Cproject/grid_shapes/corner_cell_0_1.csv
Corners for cell (0, 2) have been successfully exported to /home/jovyan/Cproject/grid_shapes/corner_cell_0_2.csv
Corners for cell (0, 3) have been successfully exported to /home/jovyan/Cproject/grid_shapes/corner_cell_0_3.csv
Corners for cell (0, 4) have been successfully exported to /home/jovyan/Cproject/grid_shapes/corner_cell_0_4.csv
Corners for cell (0, 5) have been successfully exported to /home/jovyan/Cproject/grid_shapes/corner_cell_0_5.csv
Corners for cell (0, 6) have been successfully exported to /home/jovyan/Cproject/grid_shapes/corner_cell_0_6.csv
Corners for cell (0, 7) have been successfully exported to /home/jovyan/Cproject/grid_shapes/corner_cell_0_7.csv
Corners for cell (0, 8) have been successfully exported to /home/jovyan/Cproject/grid_shapes/cor